In [ ]:
import numpy as np

In [31]:
array = np.array([[1,1,1],[2,2,2],[3,3,3]])

In [39]:
array

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

In [1]:
import wandb

# ---- 配置信息 ----
run_id = "d04p82m0"
entity_name = "315229706-xi-an-jiaotong-university-"
project_name = "FlowGRPO"

In [2]:
# ---- 用 API 获取历史 ----
api = wandb.Api()
api_run = api.run(f"{project_name}/{run_id}")

history = api_run.history()   # pandas.DataFrame

wandb: Currently logged in as: 315229706 (315229706-xi-an-jiaotong-university-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
history.keys()

Index(['reward_std_mean', 'loss', 'epoch', 'eval_reward_consistency_score',
       'policy_loss', 'approx_kl', '_runtime', 'images', '_step', 'clipfrac',
       'group_size', 'inner_epoch', 'reward_avg', 'trained_prompt_num',
       'clipfrac_gt_one', 'reward_consistency_score', '_timestamp',
       'clipfrac_lt_one', 'eval_reward_avg', 'reward_ori_avg',
       'zero_std_ratio', 'eval_images'],
      dtype='object')

In [12]:
history['epoch']

0       0
1       0
2       0
3       0
4       0
       ..
495    43
496    43
497    44
498    44
499    44
Name: epoch, Length: 500, dtype: int64

In [9]:
history['_step']

0        11
1        12
2        13
3        14
4        17
       ... 
495    2107
496    2108
497    2115
498    2118
499    2119
Name: _step, Length: 500, dtype: int64